In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
mnist_dataset, mnist_info=tfds.load('mnist', with_info=True,as_supervised=True,shuffle_files=True)
mnist_train,mnist_test=mnist_dataset['train'],mnist_dataset['test']

num_validation_samples=0.1*mnist_info.splits['train'].num_examples
num_validation_samples=tf.cast(num_validation_samples,tf.int64)

num_test_samples=mnist_info.splits['test'].num_examples
num_test_samples=tf.cast(num_test_samples,tf.int64)


def scale(image,label):
    image=tf.cast(image,tf.float32)
    image/=255.
    return image,label

scaled_train_and_validation_data=mnist_train.map(scale)
test_data=mnist_test.map(scale)
BUFFER_SIZE=10000
shuffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data=shuffled_train_and_validation_data.take(num_validation_samples)
train_data=shuffled_train_and_validation_data.skip(num_validation_samples)

Batch_Size=100
train_data=train_data.batch(Batch_Size)
validation_data=validation_data.batch(num_validation_samples)
test_data=test_data.batch(num_test_samples)

validation_inputs, validation_targets=next(iter(validation_data))

# MODEL

In [3]:
input_size=784
output_size=10
hidden_layer_size=200

model=tf.keras.Sequential([
                    tf.keras.layers.Flatten(input_shape=(28,28,1)),
                    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                    tf.keras.layers.Dense(hidden_layer_size,activation='tanh'),
                    tf.keras.layers.Dense(output_size,activation='softmax')
    
                            ])

In [4]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [5]:
NUM_EPOCHS=5
model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/5
540/540 - 10s - loss: 0.2582 - accuracy: 0.9231 - val_loss: 0.1339 - val_accuracy: 0.9575
Epoch 2/5
540/540 - 11s - loss: 0.1014 - accuracy: 0.9698 - val_loss: 0.0822 - val_accuracy: 0.9773
Epoch 3/5
540/540 - 12s - loss: 0.0669 - accuracy: 0.9792 - val_loss: 0.0681 - val_accuracy: 0.9805
Epoch 4/5
540/540 - 14s - loss: 0.0503 - accuracy: 0.9843 - val_loss: 0.0670 - val_accuracy: 0.9793
Epoch 5/5
540/540 - 10s - loss: 0.0379 - accuracy: 0.9886 - val_loss: 0.0423 - val_accuracy: 0.9893


In [7]:
test_loss, test_acc=model.evaluate(test_data)

1/1 [==============================] - 2s 2s/step - loss: 0.0741 - accuracy: 0.9781


In [8]:
print(test_loss, test_acc)

0.07411142438650131 0.9781000018119812
